### Search

In [32]:
import sys
import json
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from collections import Counter
import collections

In [33]:
import pandas as pd

In [34]:
mitodata = pd.read_csv("mitodata.csv", names = ["PMID","patient","disease","icd11","description","context","other"])
mitodata.head()

,PMID,patient,disease,icd11,description,context,other
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PMID,patient,disease,icd-11,description,context,NaN
2,11808885,NaN,barth,MG43.30,Feeding problem of infant,initial feeding difficulties,NaN
3,11808885,NaN,barth,KB08.2,Congenital hypotonia,hypotonic during infancy,NaN
4,11808885,NaN,barth,FB83.0Y,Other specified osteopenia,hypotonic during infancy,NaN


In [35]:
PMID = mitodata["PMID"][2:]
disease = mitodata["disease"][2:]
icd11 = mitodata["icd11"][2:]
description = mitodata["description"][2:]
context = mitodata["PMID"][2:]

In [50]:
MITO =[]
for a,b,c,d,e in zip(PMID,disease,icd11,description,context):
    
    imito  = collections.defaultdict(lambda : 'Key Not found')
    
    for lt in [a,b,c,d,e]:
        if lt == "NaN":
            print("found")
            lt = "Key Not found"
        
    imito.update({"PMID":a,\
               "disease":b,\
               "icd11":c,\
               "description":d,\
               "context":e})
    MITO.append(imito)
    
with open("mito.json",'w') as fw:
    json.dump(MITO,fw)

In [47]:
len(MITO)

1130

In [49]:
es = Elasticsearch(timeout=300)
k = 0
Data = []
for item in MITO:
        
        pmids = []
        
        entity = item['description']
        
        
        s = Search(using=es, index="icd11")\
                            .params(request_timeout=300)\
                            .query("match_phrase",tree=entity)
        
        
        
        for hit in s.scan():
                
                ID = str(hit.id)
                
                name = str(hit.name)
                root = str(hit.root)
                tree = str(hit.tree)
                defn = str(hit.definition)
                degree =str(hit.degree)
                childs = str(hit.childs)
            
                Data.append({"ID":ID,\
                             "name":name,\
                             "desc": item['description'],\
                             "context": item['context'],\
                             "root":root,\
                             "tree":tree,\
                             "defn":defn,\
                             "degree": degree,\
                             "childs": childs})      
        
        k = k +1
        if k%10 == 0:
                print(k,'entity counted!')
                
                
                
            

GET http://localhost:9200/icd11/_search?scroll=5m&size=1000 [status:500 request:0.006s]


10 entity counted!
20 entity counted!
30 entity counted!


TransportError: TransportError(500, 'json_parse_exception', "Non-standard token 'NaN': enable JsonParser.Feature.ALLOW_NON_NUMERIC_NUMBERS to allow\n at [Source: org.elasticsearch.transport.netty4.ByteBufStreamInput@6d5ef81e; line: 1, column: 37]")

In [45]:
Data

[{'ID': '791667961',
  'childs': '[]',
  'context': '11808885',
  'defn': 'Key Not found',
  'degree': '4',
  'desc': 'Fatigue',
  'name': 'Heat fatigue, transient',
  'root': 'Injury, poisoning or certain other consequences of external causes',
  'tree': 'Injury, poisoning or certain other consequences of external causes.Other or unspecified effects of external causes.Effects of heat.Heat fatigue, transient'},
 {'ID': '1696155577',
  'childs': '[]',
  'context': '11808885',
  'defn': 'Key Not found',
  'degree': '4',
  'desc': 'Fatigue',
  'name': 'Fatigue consumption disorder (TM1)',
  'root': 'Traditional Medicine conditions - Module I',
  'tree': 'Traditional Medicine conditions - Module I.Traditional medicine disorders (TM1).External contraction disorders (TM1).Fatigue consumption disorder (TM1)'},
 {'ID': '1116329731',
  'childs': '[]',
  'context': '11808885',
  'defn': 'Key Not found',
  'degree': '4',
  'desc': 'Fatigue',
  'name': 'Auditory fatigue deafness',
  'root': 'Disea